In [ ]:
import pandas as pd

In [ ]:
df1 = pd.read_csv("/content/combined_data_2024.csv")

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 554119 entries, 0 to 554118
Data columns (total 16 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   YEAR             554119 non-null  int64  
 1   MONTH            554119 non-null  int64  
 2   DAY              554119 non-null  int64  
 3   HOUR             554119 non-null  int64  
 4   MINUTE           554119 non-null  int64  
 5   SHIFT            554119 non-null  object 
 6   METHOD           554119 non-null  object 
 7   OFFENSE          554119 non-null  object 
 8   BLOCK            554119 non-null  object 
 9   WARD             554119 non-null  int64  
 10  ANC              554119 non-null  object 
 11  DISTRICT         554119 non-null  int64  
 12  PSA              554119 non-null  int64  
 13  VOTING_PRECINCT  554119 non-null  object 
 14  LATITUDE         554119 non-null  float64
 15  LONGITUDE        554119 non-null  float64
dtypes: float64(2), int64(8), object(6)
mem

In [ ]:
# Chi-Square Test Between Categorical Variables

from scipy.stats import chi2_contingency

# Assuming df is your DataFrame and you want to test categorical columns with 'OFFENSE'

# List of categorical columns to test against 'OFFENSE'
categorical_cols = ['SHIFT', 'METHOD', 'BLOCK', 'ANC', 'VOTING_PRECINCT']

# Perform Chi-Square test for each categorical column with the target categorical column 'OFFENSE'
for col in categorical_cols:
    # Create a contingency table
    contingency_table = pd.crosstab(df1[col], df1['OFFENSE'])

    # Perform Chi-Square test
    chi2, p, dof, expected = chi2_contingency(contingency_table)

    # Output the result
    print(f"Chi-Square Test for {col} and OFFENSE:")
    print(f"Chi2 Statistic = {chi2}, p-value = {p}")
    print(f"Degrees of Freedom = {dof}\n")


Chi-Square Test for SHIFT and OFFENSE:
Chi2 Statistic = 38444.32607351279, p-value = 0.0
Degrees of Freedom = 14

Chi-Square Test for METHOD and OFFENSE:
Chi2 Statistic = 352127.39755209006, p-value = 0.0
Degrees of Freedom = 14

Chi-Square Test for BLOCK and OFFENSE:
Chi2 Statistic = 303698.86367193045, p-value = 0.0
Degrees of Freedom = 139944

Chi-Square Test for ANC and OFFENSE:
Chi2 Statistic = 60861.32622106574, p-value = 0.0
Degrees of Freedom = 322

Chi-Square Test for VOTING_PRECINCT and OFFENSE:
Chi2 Statistic = 70108.34340219847, p-value = 0.0
Degrees of Freedom = 1001



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# Define X (features) and y (target)
x_independent = df1.drop(columns=['OFFENSE', 'BLOCK'])  # Drop the target variable and DAY column
y_dependent = df1['OFFENSE']  # Target variable

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_dependent)

# Define categorical and numerical columns
categorical_cols = ['SHIFT', 'METHOD', 'ANC', 'VOTING_PRECINCT']  # Categorical features
numerical_cols = ['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE', 'WARD', 'DISTRICT', 'PSA', 'LATITUDE', 'LONGITUDE']  # Numerical features

# One-Hot Encode Categorical Variables using pd.get_dummies()
# drop_first=True avoids dummy variable trap
X_encoded = pd.get_dummies(x_independent, columns=categorical_cols, drop_first=True)

# Combine the encoded categorical columns with numerical columns (already included in X_encoded)

# Split the data: 90% training, 10% testing
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.05, random_state=42)

# Print shapes of the training and test sets
print(f"X_train shape: {X_train.shape}, X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}, y_test shape: {y_test.shape}")


X_train shape: (526413, 203), X_test shape: (27706, 203)
y_train shape: (526413,), y_test shape: (27706,)


In [ ]:
from sklearn.metrics import classification_report
 # Till here all then svc

In [ ]:
from xgboost import XGBClassifier

# Create and configure the XGBoost model
model_xgb = XGBClassifier(
    use_label_encoder=False,         # Disable the label encoder warning
    eval_metric='mlogloss',         # Set evaluation metric for multi-class classification
    random_state=42,                # For reproducibility
    n_estimators=500,               # Increase the number of boosting rounds
    learning_rate=0.05,             # Lower learning rate
    max_depth=6,                    # Adjust max depth (experiment with values like 3, 5, or 7)
    min_child_weight=1,             # Minimum child weight to prevent overfitting
    subsample=0.8,                  # Use 80% of the training data for each tree
    colsample_bytree=0.8,           # Use 80% of features for each tree
    scale_pos_weight=None,            # Set this if you have imbalanced classes
    tree_method='gpu_hist'
)
# Fit the model
model_xgb.fit(X_train, y_train, verbose=True)

# Make predictions
y_pred = model_xgb.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [20:39:13] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [20:39:13] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [20:40:23] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


              precision    recall  f1-score   support

           0       0.00      0.00      0.00        17
           1       0.70      0.43      0.53      1674
           2       0.44      0.03      0.06      1987
           3       0.94      0.93      0.94       140
           4       0.39      0.02      0.05      2936
           5       0.64      0.37      0.47      2518
           6       0.00      0.00      0.00       170
           7       0.73      0.99      0.84     18264

    accuracy                           0.72     27706
   macro avg       0.48      0.35      0.36     27706
weighted avg       0.66      0.72      0.64     27706



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from sklearn.linear_model import LogisticRegression

model_lr = LogisticRegression(max_iter=1000, random_state=42)
# Fit the model
model_lr.fit(X_train, y_train)

# Make predictions
y_pred = model_lr.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        17
           1       1.00      0.00      0.01      1674
           2       0.00      0.00      0.00      1987
           3       0.36      0.29      0.32       140
           4       0.00      0.00      0.00      2936
           5       0.41      0.03      0.05      2518
           6       0.00      0.00      0.00       170
           7       0.67      1.00      0.80     18264

    accuracy                           0.66     27706
   macro avg       0.30      0.16      0.15     27706
weighted avg       0.54      0.66      0.53     27706



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no pre

In [ ]:
from sklearn.svm import SVC

model_svc = SVC(random_state=42)
# Fit the model
model_svc.fit(X_train, y_train)

# Make predictions
y_pred = model_svc.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model_knn = KNeighborsClassifier()
# Fit the model
model_knn.fit(X_train, y_train)

# Make predictions
y_pred = model_knn.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        17
           1       0.19      0.12      0.15      1674
           2       0.14      0.08      0.10      1987
           3       0.61      0.71      0.66       140
           4       0.19      0.11      0.14      2936
           5       0.21      0.12      0.15      2518
           6       0.06      0.01      0.01       170
           7       0.71      0.86      0.78     18264

    accuracy                           0.61     27706
   macro avg       0.26      0.25      0.25     27706
weighted avg       0.53      0.61      0.56     27706



In [ ]:
from sklearn.naive_bayes import GaussianNB

model_naivebayes = GaussianNB()
# Fit the model
model_naivebayes.fit(X_train, y_train)

# Make predictions
y_pred = model_naivebayes.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.00      0.76      0.00        17
           1       0.44      0.39      0.41      1674
           2       0.10      0.03      0.04      1987
           3       0.63      0.73      0.68       140
           4       0.17      0.08      0.11      2936
           5       0.26      0.17      0.20      2518
           6       0.01      0.04      0.02       170
           7       0.82      0.25      0.39     18264

    accuracy                           0.22     27706
   macro avg       0.30      0.31      0.23     27706
weighted avg       0.62      0.22      0.32     27706



In [ ]:
import lightgbm as lgb

model_lgb = lgb.LGBMClassifier()
# Fit the model
model_lgb.fit(X_train, y_train)

# Make predictions
y_pred = model_lgb.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.057041 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1118
[LightGBM] [Info] Number of data points in the train set: 526413, number of used features: 203
[LightGBM] [Info] Start training from score -7.284963
[LightGBM] [Info] Start training from score -2.795891
[LightGBM] [Info] Start training from score -2.635604
[LightGBM] [Info] Start training from score -5.357828
[LightGBM] [Info] Start training from score -2.238255
[LightGBM] [Info] Start training from score -2.386793
[LightGBM] [Info] Start training from score -4.981271
[LightGBM] [Info] Start training from score -0.420934
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        17
           1       0.69 

In [ ]:
from catboost import CatBoostClassifier

model_cbc = CatBoostClassifier(silent=True, random_state=42)
# Fit the model
model_cbc.fit(X_train, y_train)

# Make predictions
y_pred = model_cbc.predict(X_test)

# Evaluate the model
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        17
           1       0.71      0.44      0.54      1674
           2       0.45      0.05      0.08      1987
           3       0.93      0.94      0.93       140
           4       0.37      0.03      0.06      2936
           5       0.64      0.37      0.47      2518
           6       0.00      0.00      0.00       170
           7       0.73      0.99      0.84     18264

    accuracy                           0.72     27706
   macro avg       0.48      0.35      0.37     27706
weighted avg       0.66      0.72      0.65     27706



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

# Define the model
model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42, tree_method='gpu_hist')

# Define the parameter grid
param_grid = {
    'n_estimators': [200, 400, 600, 800, 1000],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [6, 8, 10],
    'min_child_weight': [1, 3, 5, 7, 9],
    'subsample': [0.5, 0.6, 0.7],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'gamma': [0, 0.1, 0.2]
}

In [ ]:
# Implement GridSearchCV
grid_search_accuracy = GridSearchCV(estimator=model,
                           param_grid=param_grid,
                           scoring='accuracy',  # You can also use 'f1', 'roc_auc', etc.
                           cv=5,                # 5-fold cross-validation
                           verbose=1)

# Fit the model
grid_search_accuracy.fit(X_train, y_train)

# Best parameters and score
print("Best parameters found: ", grid_search_accuracy.best_params_)
print("Best cross-validation score: {:.2f}%".format(grid_search_accuracy.best_score_ * 100))

In [ ]:
# Implement GridSearchCV for F1 Score
grid_search_f1 = GridSearchCV(estimator=model,
                               param_grid=param_grid,
                               scoring='f1_weighted',  # Use F1 Score as the scoring metric
                               cv=5,                    # 5-fold cross-validation
                               verbose=1)

# Fit the model for F1 Score
grid_search_f1.fit(X_train, y_train)

# Best parameters and score for F1 Score
print("Best parameters found (F1 Score): ", grid_search_f1.best_params_)
print("Best F1 score: {:.2f}%".format(grid_search_f1.best_score_ * 100))

In [ ]:
# Implement GridSearchCV for Recall
grid_search_recall = GridSearchCV(estimator=model,
                                   param_grid=param_grid,
                                   scoring='recall_weighted',  # Use Recall as the scoring metric
                                   cv=5,                        # 5-fold cross-validation
                                   verbose=1)

# Fit the model for Recall
grid_search_recall.fit(X_train, y_train)

# Best parameters and score for Recall
print("Best parameters found (Recall): ", grid_search_recall.best_params_)
print("Best Recall score: {:.2f}%".format(grid_search_recall.best_score_ * 100))